In [1]:
#coding:utf-8
import pandas as pd
import time

from sklearn.metrics import log_loss
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
from scipy import stats
from scipy.stats import norm, skew #加载一些统计的库

In [2]:
def time2cov(time_):
    '''
    时间是根据天数推移，所以日期为脱敏，但是时间本身不脱敏
    :param time_: 
    :return: 
    '''
    return time.strftime("%Y-%m-%d %H:%M:%S",time.localtime(time_))

In [3]:
def pre_process(data):
    '''
    :param data: 
    :return: 
    '''

    print('item_category_list_ing')
    for i in range(3):
        data['category_%d'%(i)] = data['item_category_list'].apply(
            lambda x:x.split(";")[i] if len(x.split(";")) > i else " "
        )
    del data['item_category_list']

    print('item_property_list_ing')
    for i in range(3):
        data['property_%d'%(i)] = data['item_property_list'].apply(
            lambda x:x.split(";")[i] if len(x.split(";")) > i else " "
        )
    del data['item_property_list']

    return data

In [4]:
data = pd.read_csv('train.csv',index_col=False)
data2 = pd.read_csv('test.csv',index_col=False) 

In [5]:
data = pre_process(data)
data2 = pre_process(data2)

item_category_list_ing
item_property_list_ing
item_category_list_ing
item_property_list_ing


## 统计职业对应类别

In [22]:
data[['user_occupation_id','is_trade']].groupby(['user_occupation_id']).count()

,is_trade
user_occupation_id,
-1,964
2002,146904
2003,6418
2004,23802
2005,299999


In [28]:
data[['user_occupation_id','category_1','is_trade']].groupby(['user_occupation_id','category_1']).count()/data[['user_occupation_id','is_trade']].groupby(['user_occupation_id']).count()

is_trade
user_occupation_id category_1                   
-1                 2011981573061447208  0.023859
                   22731265849056483    0.003112
                   2436715285093487584  0.012448
                   3203673979138763595  0.001037
                   4879721024980945592  0.003112
                   509660095530134768   0.174274
                   5755694407684602296  0.211618
                   5799347067982556520  0.088174
                   7258015885215914736  0.078838
                   8277336076276184272  0.394191
                   8710739180200009128  0.009336
 2002              1968056100269760729  0.000490
                   2011981573061447208  0.018965
                   22731265849056483    0.000143
                   2436715285093487584  0.004642
                   2642175453151805566  0.005480
                   3203673979138763595  0.005228
                   4879721024980945592  0.002934
                   509660095530134768   0.147695
                   5755694407684602296  0.214119
                   5799347067982556520  0.171071
                   7258015885215914736  0.125109
                   8277336076276184272  0.288862
                   8710739180200009128  0.015262
 2003              1968056100269760729  0.000156
                   2011981573061447208  0.018386
                   2436715285093487584  0.005609
                   2642175453151805566  0.004674
                   3203673979138763595  0.004051
                   4879721024980945592  0.002960
...                                          ...
                   5755694407684602296  0.210813
                   5799347067982556520  0.117482
                   7258015885215914736  0.101901
                   8277336076276184272  0.392957
                   8710739180200009128  0.017918
 2004              1968056100269760729  0.000504
                   2011981573061447208  0.018108
                   2436715285093487584  0.004958
                   2642175453151805566  0.004916
                   3203673979138763595  0.004705
                   4879721024980945592  0.002605
                   509660095530134768   0.152508
                   5755694407684602296  0.190824
                   5799347067982556520  0.189606
                   7258015885215914736  0.118645
                   8277336076276184272  0.298756
                   8710739180200009128  0.013864
 2005              1968056100269760729  0.000337
                   2011981573061447208  0.020680
                   22731265849056483    0.000213
                   2436715285093487584  0.003727
                   2642175453151805566  0.003590
                   3203673979138763595  0.004460
                   4879721024980945592  0.002307
                   509660095530134768   0.163767
                   5755694407684602296  0.216451
                   5799347067982556520  0.138427
                   7258015885215914736  0.103687
                   8277336076276184272  0.327728
                   8710739180200009128  0.014627

[61 rows x 1 columns]

## 开始统计

In [29]:
all_data = data.append(data2)

In [30]:
occupation_click = all_data[['user_occupation_id','category_1','instance_id']].groupby(['user_occupation_id','category_1']).count()/data[['user_occupation_id','instance_id']].groupby(['user_occupation_id']).count()

In [31]:
occupation_click = occupation_click.rename(columns=({'instance_id':'occupation_click'}))

In [32]:
occupation_click = occupation_click.reset_index()

In [33]:
occupation_click

,user_occupation_id,category_1,occupation_click
0,-1,2011981573061447208,0.023859
1,-1,22731265849056483,0.003112
2,-1,2436715285093487584,0.013485
3,-1,3203673979138763595,0.001037
4,-1,4879721024980945592,0.003112
5,-1,509660095530134768,0.177386
6,-1,5755694407684602296,0.217842
7,-1,5799347067982556520,0.090249
8,-1,7258015885215914736,0.079876
9,-1,8277336076276184272,0.399378


In [34]:
occupationclickdata = pd.merge(data,occupation_click,on=['user_occupation_id','category_1'],how='left')

In [35]:
occupationclickdata2 = pd.merge(data2,occupation_click,on=['user_occupation_id','category_1'],how='left')

In [36]:
occupationclickdata2

,instance_id,item_id,item_brand_id,item_city_id,item_price_level,item_sales_level,item_collected_level,item_pv_level,user_id,user_gender_id,...,shop_score_service,shop_score_delivery,shop_score_description,category_0,category_1,category_2,property_0,property_1,property_2,occupation_click
0,2475218615076601065,2275895163219263378,7838285046767229711,7534238860363577544,7,6,8,14,4379490381248358680,0,...,0.964023,0.966550,0.955378,7908382889764677758,8277336076276184272,,2636395404473730413,7515802706813700848,302719828435032992,0.341078
1,398316874173557226,7096238490711246967,8126195666233054089,3948283326616421003,6,9,8,16,2629129219935839101,1,...,0.963794,0.964225,0.959134,7908382889764677758,5755694407684602296,,5131280576272319091,2636395404473730413,7344985833148694227,0.222989
2,6586402638209028583,7096238490711246967,8126195666233054089,3948283326616421003,6,9,8,16,1092007111346257147,0,...,0.963794,0.964225,0.959134,7908382889764677758,5755694407684602296,,5131280576272319091,2636395404473730413,7344985833148694227,0.222989
3,1040996105851528465,7096238490711246967,8126195666233054089,3948283326616421003,6,9,8,16,5353289677517820939,1,...,0.963794,0.964225,0.959134,7908382889764677758,5755694407684602296,,5131280576272319091,2636395404473730413,7344985833148694227,0.222989
4,6316278569655873454,7096238490711246967,8126195666233054089,3948283326616421003,6,9,8,16,1098833840660870025,0,...,0.963794,0.964225,0.959134,7908382889764677758,5755694407684602296,,5131280576272319091,2636395404473730413,7344985833148694227,0.219850
5,868158305045921978,7096238490711246967,8126195666233054089,3948283326616421003,6,9,8,16,158655638471934821,0,...,0.963794,0.964225,0.959134,7908382889764677758,5755694407684602296,,5131280576272319091,2636395404473730413,7344985833148694227,0.224821
6,5713520501786699854,1171140458228333440,1547983184598012595,8072963182326625214,5,6,6,15,7254617831443442890,0,...,0.976897,0.977126,0.984791,7908382889764677758,4879721024980945592,,5131280576272319091,2636395404473730413,3163265386149801264,0.003070
7,932945015407923184,7020520887593189887,8324857885829459863,1019055478500227370,7,10,11,14,7588649790698790290,1,...,0.979252,0.980538,0.985626,7908382889764677758,2011981573061447208,,5131280576272319091,1013419034389836991,5195139481388729954,0.021497
8,1919197847086752313,8143258757857432437,8324857885829459863,1019055478500227370,8,10,13,15,7745527540524595604,0,...,0.979252,0.980538,0.985626,7908382889764677758,2011981573061447208,,2636395404473730413,4621934203383159480,1243768796732904674,0.019714
9,304887065966615346,8143258757857432437,8324857885829459863,1019055478500227370,8,10,13,15,2401138934950272514,0,...,0.979252,0.980538,0.985626,7908382889764677758,2011981573061447208,,2636395404473730413,4621934203383159480,1243768796732904674,0.019714


In [37]:
occupationclickdata[['instance_id','occupation_click']].to_csv('train_occupation_click.csv',index=False) 

In [38]:
occupationclickdata2[['instance_id','occupation_click']].to_csv('test_occupation_click.csv',index=False) 